In [1]:
import torch
import torch.nn.functional as F  
from torch import optim 
from torch import nn
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import torch.nn.init as init
import random
import time
from sklearn.model_selection import KFold

### Class for the Neural Network

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes, hidden_layer_sizes, activation_function, apply_softmax = False):
        super(NN, self).__init__()

        seed = 18
        torch.manual_seed(seed)

        self.activation = activation_function
        self.apply_softmax = apply_softmax

        # Input layer
        self.input_layer = nn.Linear(input_size, hidden_layer_sizes[0])
        init.kaiming_uniform_(self.input_layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


        # Hidden layers
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_layer_sizes[i], hidden_layer_sizes[i + 1])
            for i in range(len(hidden_layer_sizes) - 1)
        ])

        for layer in self.hidden_layers:
            init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


        # Output layer
        self.output_layer = nn.Linear(hidden_layer_sizes[-1], num_classes)
        init.kaiming_uniform_(self.output_layer.weight, mode='fan_in', nonlinearity=activation_function.__name__)


    def forward(self, x):
        x = x.float()
        x = self.activation(self.input_layer(x))

        # Process through hidden layers
        for layer in self.hidden_layers:
            x = self.activation(layer(x))

        if self.apply_softmax:
            x = F.softmax(self.output_layer(x), dim=1)
        else:
            x = self.output_layer(x)

        return x
    

### Function to train the model

In [3]:
def train_model(model, train_loader, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        total_loss = 0.0  # Initialize total loss for the epoch
        num_batches = len(train_loader)

        for batch_idx, (data, targets) in enumerate(train_loader):
            data = data.reshape(data.shape[0], -1)

            scores = model(data)
            loss = criterion(scores, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()  # Accumulate the batch loss

        average_loss = total_loss / num_batches
        print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}")
    

### Function to calculate the accuracy

In [4]:
def check_accuracy(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:

            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


### Function to split the data into train and test

In [5]:
def train_test_split(data: pd.DataFrame, target_label : str, test_size=0.2, return_torch=None):
        
    # split the data into train and test
    train = data.sample(frac=(1-test_size),random_state=200)
    test = data.drop(train.index)
    
    # split the train and test into X and Y
    train_X = train.drop([target_label], axis=1).values
    train_Y = train[target_label].values
    test_X = test.drop([target_label], axis=1).values
    test_Y = test[target_label].values
    
    if return_torch:
        train_X = torch.tensor(train_X)
        train_Y = torch.tensor(train_Y)
        test_X = torch.tensor(test_X)
        test_Y = torch.tensor(test_Y)
    
    return train_X, train_Y, test_X, test_Y

## Function for Grid Search

In [31]:
def grid_search_cv(hidden_layer_sizes_list, activation_functions, learning_rates, batch_sizes, num_epochs_list, train_loader, k_folds=5, use_scaling=True):
    best_accuracy = 0.0
    best_combination = None
    results = []

    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    if use_scaling:
        scaler = StandardScaler()
        scaler.fit(train_loader.dataset.tensors[0].numpy())

    for hidden_layer_sizes in hidden_layer_sizes_list:
        for activation_function in activation_functions:
            for learning_rate in learning_rates:
                for batch_size in batch_sizes:
                    for num_epochs in num_epochs_list:

                        fold_accuracies = []
                        fold_training_times = []

                        for train_index, test_index in kf.split(train_loader.dataset):
                            X_train, X_test = train_loader.dataset.tensors[0][train_index], train_loader.dataset.tensors[0][test_index]
                            y_train, y_test = train_loader.dataset.tensors[1][train_index], train_loader.dataset.tensors[1][test_index]

                            model = NN(input_size=train_loader.dataset.tensors[0].shape[1],
                                       num_classes=num_classes,
                                       hidden_layer_sizes=hidden_layer_sizes,
                                       activation_function=activation_function)
                            criterion = nn.CrossEntropyLoss()
                            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

                        
                            if use_scaling:
                                X_train_scaled = torch.tensor(scaler.transform(X_train.numpy()))
                                X_test_scaled = torch.tensor(scaler.transform(X_test.numpy()))
                            else:
                                X_train_scaled, X_test_scaled = X_train, X_test

                            train_start_time = time.time()
                            train_model(model, DataLoader(TensorDataset(X_train_scaled, y_train), batch_size=batch_size, shuffle=True), optimizer, criterion, num_epochs)
                            train_end_time = time.time()
                            fold_training_times.append(train_end_time - train_start_time)

                            accuracy_test = check_accuracy(DataLoader(TensorDataset(X_test_scaled, y_test), batch_size=batch_size, shuffle=False), model)
                            fold_accuracies.append(accuracy_test.item())

                        avg_accuracy = np.mean(fold_accuracies)
                        avg_training_time = np.mean(fold_training_times)

                        result = {
                            'Hidden Layer Sizes': hidden_layer_sizes,
                            'Activation Function': activation_function.__name__,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size,
                            'Number of Epochs': num_epochs,
                            'Average Accuracy': avg_accuracy,
                            'Average Training Time': avg_training_time
                        }

                        results.append(result)

                        if avg_accuracy > best_accuracy:
                            best_accuracy = avg_accuracy
                            best_combination = result

    results_df = pd.DataFrame(results)
    return results_df, best_accuracy, best_combination

## Testing the model on the wine quality dataset

#### Loading the dataset

In [8]:
wine_quality = pd.read_csv('./preprocessed-datasets/wine_quality_prepro.csv', index_col=0)
wine_quality.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


#### Splitting the dataset into training and testing sets, converting to PyTorch tensors and creating PyTorch DataLoaders

In [9]:
train_X, train_Y, test_X, test_Y = train_test_split(wine_quality, "class", return_torch=True)

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=False)

dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)


#### Creating the model, training and testing

In [10]:
input_size = train_X.shape[1] # number of features in wine quality dataset
num_classes = 10 # 10 classes in wine quality dataset
learning_rate = 0.01
batch_size = 64
num_epochs = 10
hidden_layer_sizes = [25,30]
activation_function = F.tanh

model = NN(input_size=train_X.shape[1], num_classes=num_classes, hidden_layer_sizes=hidden_layer_sizes, activation_function=activation_function)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_model(model, train_loader, optimizer, criterion, num_epochs)

print(f"Accuracy on training set: {check_accuracy(train_loader, model)}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)}")

Epoch 1/10, Average Loss: 1.3338039038371454
Epoch 2/10, Average Loss: 1.2913920031003425
Epoch 3/10, Average Loss: 1.2882495277498398
Epoch 4/10, Average Loss: 1.287879331711611
Epoch 5/10, Average Loss: 1.2764818719559652
Epoch 6/10, Average Loss: 1.273354259736699
Epoch 7/10, Average Loss: 1.2613155139735872
Epoch 8/10, Average Loss: 1.259151681069216
Epoch 9/10, Average Loss: 1.2574602010791287
Epoch 10/10, Average Loss: 1.2471537063458213
Accuracy on training set: 0.44132357835769653
Accuracy on test set: 0.4470588266849518


## Testing the model on the congressional voting dataset

#### Loading the dataset

In [13]:
cong_voting = pd.read_csv('./preprocessed-datasets/CongressionVoting_prepro.csv')
# encode class value democrat as 1 and republican as 0
cong_voting['class'] = cong_voting['class'].map({'democrat': 1, 'republican': 0})
cong_voting.head()

,ID,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-crporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa,class
0,140,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
1,383,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1
2,201,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
3,297,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0
4,309,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0


#### Splitting the dataset into training and testing sets, converting to PyTorch tensors and creating PyTorch DataLoaders

In [14]:
train_X, train_Y, test_X, test_Y = train_test_split(cong_voting, "class", return_torch=True)

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=False)

dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)

#### Creating the model, training and testing

In [15]:
input_size = train_X.shape[1] # number of features in congr voting dataset
num_classes = 2 # 2 classes in congr voting dataset
learning_rate = 0.01
batch_size = 64
num_epochs = 10
hidden_layer_sizes = [25,30]
activation_function = F.tanh

model = NN(input_size=train_X.shape[1], num_classes=num_classes, hidden_layer_sizes=hidden_layer_sizes, activation_function=activation_function)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_model(model, train_loader, optimizer, criterion, num_epochs)

print(f"Accuracy on training set: {check_accuracy(train_loader, model)}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)}")

Epoch 1/10, Average Loss: 0.8204039831956228
Epoch 2/10, Average Loss: 0.6987561782201132
Epoch 3/10, Average Loss: 0.6630964974562327
Epoch 4/10, Average Loss: 0.5736258924007416
Epoch 5/10, Average Loss: 0.4424207905928294
Epoch 6/10, Average Loss: 0.3960420439640681
Epoch 7/10, Average Loss: 0.29747725029786426
Epoch 8/10, Average Loss: 0.2279346063733101
Epoch 9/10, Average Loss: 0.22333091124892235
Epoch 10/10, Average Loss: 0.3738237793246905
Accuracy on training set: 0.959770143032074
Accuracy on test set: 0.8604651093482971


## Testing the model on bank marketing dataset

#### Loading and preparing dataset

In [16]:
bank_marketing = pd.read_csv('./preprocessed-datasets/bank_marketing_prepro.csv')

In [17]:
column_to_move = 'class'

# Move class to the last index
columns = [col for col in bank_marketing.columns if col != column_to_move] + [column_to_move]
bank_marketing = bank_marketing[columns]

bank_marketing.drop('Unnamed: 0', axis=1,inplace=True)

bank_marketing.head()

,age,default,housing,loan,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,...,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,poutcome_failure,poutcome_nonexistent,poutcome_success,class
0,56,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
1,57,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
2,37,0.0,1.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
3,40,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
4,56,0.0,0.0,1.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0


In [18]:
bank_marketing.columns

Index(['age', 'default', 'housing', 'loan', 'campaign', 'pdays', 'previous',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'job_blue-collar', 'job_management', 'job_other',
       'job_self-employed', 'job_serivces', 'job_technician',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'poutcome_failure', 'poutcome_nonexistent',
       'poutcome_success', 'class'],
      dtype='object')

#### Splitting the dataset into training and testing sets, converting to PyTorch tensors and creating PyTorch DataLoaders

In [19]:
train_X, train_Y, test_X, test_Y = train_test_split(bank_marketing, "class", return_torch=True)

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=False)

dataset = TensorDataset(test_X, test_Y)
test_loader = DataLoader(dataset, batch_size=32, shuffle=False)

#### Creating the model, training and testing

In [20]:
input_size = train_X.shape[1] 
num_classes = 2 
learning_rate = 0.01
batch_size = 64
num_epochs = 10
hidden_layer_sizes = [25,30]
activation_function = F.tanh

model = NN(input_size=train_X.shape[1], num_classes=num_classes, hidden_layer_sizes=hidden_layer_sizes, activation_function=activation_function)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_model(model, train_loader, optimizer, criterion, num_epochs)

print(f"Accuracy on training set: {check_accuracy(train_loader, model)}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)}")

Epoch 1/10, Average Loss: 0.33090905211142546
Epoch 2/10, Average Loss: 0.32768635471686
Epoch 3/10, Average Loss: 0.327213676422111
Epoch 4/10, Average Loss: 0.3280542928255299
Epoch 5/10, Average Loss: 0.32849315527139356
Epoch 6/10, Average Loss: 0.32820502676937763
Epoch 7/10, Average Loss: 0.3284298519956545
Epoch 8/10, Average Loss: 0.3284010060674068
Epoch 9/10, Average Loss: 0.3283500720249507
Epoch 10/10, Average Loss: 0.3283919007888118
Accuracy on training set: 0.8986949920654297
Accuracy on test set: 0.8928138017654419


# Test Grid search over all three datasets

It was not possible for us to iterate over all datasets because of the large amount of runtime

### Bank Marketing

In [41]:
hidden_layer_sizes_list = [[5],[10],[25, 30], [20, 25, 30]]
activation_functions = [F.tanh, F.relu, F.sigmoid]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [64]
num_epochs_list = [10]

dataset = bank_marketing

train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True)

train_data = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

test_data = TensorDataset(test_X, test_Y)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

input_size = train_X.shape[1]

num_classes = 2

grid_results_bank, best_accuracy, best_combination = grid_search_cv(
    hidden_layer_sizes_list, activation_functions, learning_rates, batch_sizes, num_epochs_list, train_loader,
    k_folds=5, use_scaling=True)

grid_results_bank['dataset'] = 'bank_marketing'

Epoch 1/10, Average Loss: 0.31337055091936034
Epoch 2/10, Average Loss: 0.2875420349532539
Epoch 3/10, Average Loss: 0.2865312381746989
Epoch 4/10, Average Loss: 0.28597566803682195
Epoch 5/10, Average Loss: 0.28572057173090076
Epoch 6/10, Average Loss: 0.2839459879052581
Epoch 7/10, Average Loss: 0.283088373110711
Epoch 8/10, Average Loss: 0.2823086593083097
Epoch 9/10, Average Loss: 0.28205011374525074
Epoch 10/10, Average Loss: 0.2812420981890947
Epoch 1/10, Average Loss: 0.31708944022872493
Epoch 2/10, Average Loss: 0.28990169195006193
Epoch 3/10, Average Loss: 0.28756402429301764
Epoch 4/10, Average Loss: 0.28596305758556695
Epoch 5/10, Average Loss: 0.2846016995970485
Epoch 6/10, Average Loss: 0.28440924994285827
Epoch 7/10, Average Loss: 0.28351520698119714
Epoch 8/10, Average Loss: 0.2830812868309542
Epoch 9/10, Average Loss: 0.282132304463427
Epoch 10/10, Average Loss: 0.2820775376325383
Epoch 1/10, Average Loss: 0.3095183370508326
Epoch 2/10, Average Loss: 0.28360716926241386

Epoch 1/10, Average Loss: 0.31277498975396156
Epoch 2/10, Average Loss: 0.2845193466479049
Epoch 3/10, Average Loss: 0.28127829169908775
Epoch 4/10, Average Loss: 0.2812378028932127
Epoch 5/10, Average Loss: 0.2809798424719896
Epoch 6/10, Average Loss: 0.28037277737505806
Epoch 7/10, Average Loss: 0.2794966722648699
Epoch 8/10, Average Loss: 0.2796581468897537
Epoch 9/10, Average Loss: 0.2800517247320668
Epoch 10/10, Average Loss: 0.27919057825381316
Epoch 1/10, Average Loss: 0.313243238146878
Epoch 2/10, Average Loss: 0.2833644069586564
Epoch 3/10, Average Loss: 0.28203170952721707
Epoch 4/10, Average Loss: 0.28050482381605407
Epoch 5/10, Average Loss: 0.27983622365061517
Epoch 6/10, Average Loss: 0.2797870781381153
Epoch 7/10, Average Loss: 0.27940834550530586
Epoch 8/10, Average Loss: 0.27912001870572567
Epoch 9/10, Average Loss: 0.2791865635663271
Epoch 10/10, Average Loss: 0.2787827271425608
Epoch 1/10, Average Loss: 0.43671484132414884
Epoch 2/10, Average Loss: 0.3191447730347948

Epoch 2/10, Average Loss: 0.29748113297866385
Epoch 3/10, Average Loss: 0.2921111363085728
Epoch 4/10, Average Loss: 0.2896801284649997
Epoch 5/10, Average Loss: 0.28826628059365794
Epoch 6/10, Average Loss: 0.2875780083275246
Epoch 7/10, Average Loss: 0.28711446707399146
Epoch 8/10, Average Loss: 0.2867509770993758
Epoch 9/10, Average Loss: 0.2864288611210955
Epoch 10/10, Average Loss: 0.28618141301699634
Epoch 1/10, Average Loss: 0.360891500940022
Epoch 2/10, Average Loss: 0.2917019564260557
Epoch 3/10, Average Loss: 0.2858807332994579
Epoch 4/10, Average Loss: 0.2836611346676222
Epoch 5/10, Average Loss: 0.28215930274533996
Epoch 6/10, Average Loss: 0.2812684720508682
Epoch 7/10, Average Loss: 0.2807519120860447
Epoch 8/10, Average Loss: 0.2803458370702359
Epoch 9/10, Average Loss: 0.2800918926204582
Epoch 10/10, Average Loss: 0.2795037578212694
Epoch 1/10, Average Loss: 0.36122039727215627
Epoch 2/10, Average Loss: 0.29276028752905653
Epoch 3/10, Average Loss: 0.28711569233282097
E

Epoch 3/10, Average Loss: 0.28685915737765505
Epoch 4/10, Average Loss: 0.2839388343351848
Epoch 5/10, Average Loss: 0.2818175999321116
Epoch 6/10, Average Loss: 0.28060049910857837
Epoch 7/10, Average Loss: 0.2795953510636554
Epoch 8/10, Average Loss: 0.27852229045215743
Epoch 9/10, Average Loss: 0.27771854411485125
Epoch 10/10, Average Loss: 0.2772128963752568
Epoch 1/10, Average Loss: 1.0585615931494723
Epoch 2/10, Average Loss: 0.811826693490871
Epoch 3/10, Average Loss: 0.6524092629551888
Epoch 4/10, Average Loss: 0.5463773888146993
Epoch 5/10, Average Loss: 0.47263855696881857
Epoch 6/10, Average Loss: 0.42036082834294697
Epoch 7/10, Average Loss: 0.3829146822896397
Epoch 8/10, Average Loss: 0.3557311277655722
Epoch 9/10, Average Loss: 0.3362859576506522
Epoch 10/10, Average Loss: 0.3223726067294195
Epoch 1/10, Average Loss: 1.0597437540593657
Epoch 2/10, Average Loss: 0.8163202409316035
Epoch 3/10, Average Loss: 0.657083508386774
Epoch 4/10, Average Loss: 0.5507667052369674
Epoc

Epoch 4/10, Average Loss: 0.3852184661890118
Epoch 5/10, Average Loss: 0.3507969151567487
Epoch 6/10, Average Loss: 0.33357325268601906
Epoch 7/10, Average Loss: 0.3236898967941988
Epoch 8/10, Average Loss: 0.31724482543260146
Epoch 9/10, Average Loss: 0.3124225891367035
Epoch 10/10, Average Loss: 0.3085797409734969
Epoch 1/10, Average Loss: 0.9237313836234288
Epoch 2/10, Average Loss: 0.621143023004231
Epoch 3/10, Average Loss: 0.45986722670133834
Epoch 4/10, Average Loss: 0.3854689772412615
Epoch 5/10, Average Loss: 0.3511494987653297
Epoch 6/10, Average Loss: 0.3339532843058549
Epoch 7/10, Average Loss: 0.3241693391961959
Epoch 8/10, Average Loss: 0.3177597837757717
Epoch 9/10, Average Loss: 0.31307486050481936
Epoch 10/10, Average Loss: 0.30932336445282965
Epoch 1/10, Average Loss: 0.9247061018515559
Epoch 2/10, Average Loss: 0.621726705828338
Epoch 3/10, Average Loss: 0.4592314008804201
Epoch 4/10, Average Loss: 0.38425769432655815
Epoch 5/10, Average Loss: 0.3497707569266407
Epoc

Epoch 5/10, Average Loss: 0.2818562227111418
Epoch 6/10, Average Loss: 0.2808187132864033
Epoch 7/10, Average Loss: 0.280345064072499
Epoch 8/10, Average Loss: 0.2797199697821464
Epoch 9/10, Average Loss: 0.2788115340348297
Epoch 10/10, Average Loss: 0.27915802205578216
Epoch 1/10, Average Loss: 0.3082456316762758
Epoch 2/10, Average Loss: 0.28909493662706276
Epoch 3/10, Average Loss: 0.28616463392972946
Epoch 4/10, Average Loss: 0.28492739530804667
Epoch 5/10, Average Loss: 0.28410923294723034
Epoch 6/10, Average Loss: 0.281822410602014
Epoch 7/10, Average Loss: 0.28148704374805816
Epoch 8/10, Average Loss: 0.2813910843388548
Epoch 9/10, Average Loss: 0.2799620909019581
Epoch 10/10, Average Loss: 0.27901763342393254
Epoch 1/10, Average Loss: 0.3001644851151601
Epoch 2/10, Average Loss: 0.2823716169313609
Epoch 3/10, Average Loss: 0.28084630144814265
Epoch 4/10, Average Loss: 0.2775025319995232
Epoch 5/10, Average Loss: 0.27705263775022865
Epoch 6/10, Average Loss: 0.2773539421772494
E

Epoch 6/10, Average Loss: 0.2782750453040438
Epoch 7/10, Average Loss: 0.2783201151862017
Epoch 8/10, Average Loss: 0.277395422941128
Epoch 9/10, Average Loss: 0.2766224793836619
Epoch 10/10, Average Loss: 0.276115850265165
Epoch 1/10, Average Loss: 0.29888301263156447
Epoch 2/10, Average Loss: 0.2821301704869398
Epoch 3/10, Average Loss: 0.28105289584875687
Epoch 4/10, Average Loss: 0.28005246458195365
Epoch 5/10, Average Loss: 0.2785657098498738
Epoch 6/10, Average Loss: 0.2788655589723471
Epoch 7/10, Average Loss: 0.2773298703459571
Epoch 8/10, Average Loss: 0.27672175610962424
Epoch 9/10, Average Loss: 0.2761284168267134
Epoch 10/10, Average Loss: 0.27462813265404656
Epoch 1/10, Average Loss: 0.34961233389319724
Epoch 2/10, Average Loss: 0.29249164445813997
Epoch 3/10, Average Loss: 0.28691934938859015
Epoch 4/10, Average Loss: 0.2837471750935594
Epoch 5/10, Average Loss: 0.28167894804506627
Epoch 6/10, Average Loss: 0.28025935977263355
Epoch 7/10, Average Loss: 0.2793770020339385


Epoch 7/10, Average Loss: 0.28501818945757973
Epoch 8/10, Average Loss: 0.2845666927909388
Epoch 9/10, Average Loss: 0.28446683264111433
Epoch 10/10, Average Loss: 0.2839233852308063
Epoch 1/10, Average Loss: 0.33885040029304697
Epoch 2/10, Average Loss: 0.2837657724480027
Epoch 3/10, Average Loss: 0.2808341540360046
Epoch 4/10, Average Loss: 0.27971201048241656
Epoch 5/10, Average Loss: 0.2791405305157877
Epoch 6/10, Average Loss: 0.2788309653660337
Epoch 7/10, Average Loss: 0.2780723581723507
Epoch 8/10, Average Loss: 0.27778575777356485
Epoch 9/10, Average Loss: 0.27759917290176
Epoch 10/10, Average Loss: 0.277307856918394
Epoch 1/10, Average Loss: 0.3398612934408836
Epoch 2/10, Average Loss: 0.2847421075967909
Epoch 3/10, Average Loss: 0.28140952644272915
Epoch 4/10, Average Loss: 0.280561108829327
Epoch 5/10, Average Loss: 0.28005733621279594
Epoch 6/10, Average Loss: 0.2796735155206282
Epoch 7/10, Average Loss: 0.27925632765643227
Epoch 8/10, Average Loss: 0.2790534519997326
Epoc

Epoch 8/10, Average Loss: 0.27064423208821164
Epoch 9/10, Average Loss: 0.2692965394952922
Epoch 10/10, Average Loss: 0.26785884837357743
Epoch 1/10, Average Loss: 0.5463524185193395
Epoch 2/10, Average Loss: 0.35916824758197496
Epoch 3/10, Average Loss: 0.31837219763004665
Epoch 4/10, Average Loss: 0.30524960417047287
Epoch 5/10, Average Loss: 0.2998011428681971
Epoch 6/10, Average Loss: 0.2967558941143809
Epoch 7/10, Average Loss: 0.29452899054994863
Epoch 8/10, Average Loss: 0.2928321693562767
Epoch 9/10, Average Loss: 0.2912903424460911
Epoch 10/10, Average Loss: 0.29003457654520726
Epoch 1/10, Average Loss: 0.5435493223152115
Epoch 2/10, Average Loss: 0.3591433708167192
Epoch 3/10, Average Loss: 0.31895815400243965
Epoch 4/10, Average Loss: 0.30603795393722727
Epoch 5/10, Average Loss: 0.3007686707796981
Epoch 6/10, Average Loss: 0.29763782953897727
Epoch 7/10, Average Loss: 0.29554767384040126
Epoch 8/10, Average Loss: 0.2937841608360844
Epoch 9/10, Average Loss: 0.29240349779646

Epoch 9/10, Average Loss: 0.28820303017363963
Epoch 10/10, Average Loss: 0.2865600766234317
Epoch 1/10, Average Loss: 0.4057074073448922
Epoch 2/10, Average Loss: 0.32858994653792056
Epoch 3/10, Average Loss: 0.3132337667952174
Epoch 4/10, Average Loss: 0.3053392859410892
Epoch 5/10, Average Loss: 0.3001116492866891
Epoch 6/10, Average Loss: 0.29650498024251276
Epoch 7/10, Average Loss: 0.2935042841703568
Epoch 8/10, Average Loss: 0.29118150092878387
Epoch 9/10, Average Loss: 0.28917756108838377
Epoch 10/10, Average Loss: 0.2875164177051736
Epoch 1/10, Average Loss: 0.3985758064658318
Epoch 2/10, Average Loss: 0.32546930072955715
Epoch 3/10, Average Loss: 0.31063040711202666
Epoch 4/10, Average Loss: 0.303031941984319
Epoch 5/10, Average Loss: 0.2980683179011623
Epoch 6/10, Average Loss: 0.29464538194673157
Epoch 7/10, Average Loss: 0.2919686854030322
Epoch 8/10, Average Loss: 0.28987857748220847
Epoch 9/10, Average Loss: 0.28808615029364537
Epoch 10/10, Average Loss: 0.286600335184665

In [42]:
grid_results_bank

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Average Accuracy,Training Time,dataset
0,[5],tanh,0.0100,64,10,0.898543,10.444025,bank_marketing
1,[5],tanh,0.0010,64,10,0.898452,10.106878,bank_marketing
2,[5],tanh,0.0001,64,10,0.883520,12.306125,bank_marketing
3,[5],relu,0.0100,64,10,0.897997,10.063668,bank_marketing
4,[5],relu,0.0010,64,10,0.898270,10.309429,bank_marketing
5,[5],relu,0.0001,64,10,0.894537,14.351665,bank_marketing
6,[5],sigmoid,0.0100,64,10,0.899545,13.074883,bank_marketing
7,[5],sigmoid,0.0010,64,10,0.899423,9.855753,bank_marketing
8,[5],sigmoid,0.0001,64,10,0.888407,13.924242,bank_marketing
9,[10],tanh,0.0100,64,10,0.897147,9.866659,bank_marketing


### Wine quality

In [43]:
hidden_layer_sizes_list = [[5],[10],[25, 30], [20, 25, 30]]
activation_functions = [F.tanh, F.relu, F.sigmoid]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [64]
num_epochs_list = [10]

dataset = wine_quality

train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True)

train_data = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

test_data = TensorDataset(test_X, test_Y)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

input_size = train_X.shape[1]

num_classes = 10

grid_results_wine, best_accuracy, best_combination = grid_search_cv(
    hidden_layer_sizes_list, activation_functions, learning_rates, batch_sizes, num_epochs_list, train_loader,
    k_folds=5, use_scaling=True)

grid_results_wine['dataset'] = 'wine_quality'

Epoch 1/10, Average Loss: 1.9640880859815157
Epoch 2/10, Average Loss: 1.323743330515348
Epoch 3/10, Average Loss: 1.137873377249791
Epoch 4/10, Average Loss: 1.0958091680820172
Epoch 5/10, Average Loss: 1.0786131079380328
Epoch 6/10, Average Loss: 1.068297107403095
Epoch 7/10, Average Loss: 1.0614138786609357
Epoch 8/10, Average Loss: 1.0555294651251572
Epoch 9/10, Average Loss: 1.051697132220635
Epoch 10/10, Average Loss: 1.0490676925732538
Epoch 1/10, Average Loss: 1.958414937899663
Epoch 2/10, Average Loss: 1.3395370831856361
Epoch 3/10, Average Loss: 1.1546646815079908
Epoch 4/10, Average Loss: 1.1115979964916523
Epoch 5/10, Average Loss: 1.0945900119267977
Epoch 6/10, Average Loss: 1.0824062319902272
Epoch 7/10, Average Loss: 1.0738110854075504
Epoch 8/10, Average Loss: 1.0637168957636907
Epoch 9/10, Average Loss: 1.0559407921937796
Epoch 10/10, Average Loss: 1.0514039167991052
Epoch 1/10, Average Loss: 1.9654830895937405
Epoch 2/10, Average Loss: 1.3331183396852933
Epoch 3/10, A

Epoch 5/10, Average Loss: 1.1124105150883015
Epoch 6/10, Average Loss: 1.0992311046673702
Epoch 7/10, Average Loss: 1.088952654141646
Epoch 8/10, Average Loss: 1.0813729552122262
Epoch 9/10, Average Loss: 1.0731652424885676
Epoch 10/10, Average Loss: 1.0688021696530856
Epoch 1/10, Average Loss: 2.077593884101281
Epoch 2/10, Average Loss: 1.3203212151160606
Epoch 3/10, Average Loss: 1.1808672180542579
Epoch 4/10, Average Loss: 1.1259993819090037
Epoch 5/10, Average Loss: 1.0984286390818083
Epoch 6/10, Average Loss: 1.0850531440514786
Epoch 7/10, Average Loss: 1.0713634225038382
Epoch 8/10, Average Loss: 1.064828109741211
Epoch 9/10, Average Loss: 1.0575788204486554
Epoch 10/10, Average Loss: 1.053063966677739
Epoch 1/10, Average Loss: 3.403772970346304
Epoch 2/10, Average Loss: 2.8042698383331297
Epoch 3/10, Average Loss: 2.414530695401705
Epoch 4/10, Average Loss: 2.158608526449937
Epoch 5/10, Average Loss: 1.9852515037243184
Epoch 6/10, Average Loss: 1.8583430070143478
Epoch 7/10, Ave

Epoch 9/10, Average Loss: 1.3711947661179762
Epoch 10/10, Average Loss: 1.326188824726985
Epoch 1/10, Average Loss: 2.5550344467163084
Epoch 2/10, Average Loss: 2.278204815204327
Epoch 3/10, Average Loss: 2.050037321677575
Epoch 4/10, Average Loss: 1.8626287350287805
Epoch 5/10, Average Loss: 1.711266592832712
Epoch 6/10, Average Loss: 1.5916351006581233
Epoch 7/10, Average Loss: 1.4978710944835956
Epoch 8/10, Average Loss: 1.424523296722999
Epoch 9/10, Average Loss: 1.3671882904492891
Epoch 10/10, Average Loss: 1.3221423809344952
Epoch 1/10, Average Loss: 2.547891202339759
Epoch 2/10, Average Loss: 2.2730434014247014
Epoch 3/10, Average Loss: 2.0471745362648597
Epoch 4/10, Average Loss: 1.861719986108633
Epoch 5/10, Average Loss: 1.7118036618599526
Epoch 6/10, Average Loss: 1.5929644144498385
Epoch 7/10, Average Loss: 1.5000507831573486
Epoch 8/10, Average Loss: 1.4274941169298612
Epoch 9/10, Average Loss: 1.3708354473114013
Epoch 10/10, Average Loss: 1.3264395347008338
Epoch 1/10, Av

Epoch 3/10, Average Loss: 2.76491866845351
Epoch 4/10, Average Loss: 2.7216953864464393
Epoch 5/10, Average Loss: 2.679950108894935
Epoch 6/10, Average Loss: 2.6394038053659292
Epoch 7/10, Average Loss: 2.599927572103647
Epoch 8/10, Average Loss: 2.5617853458111104
Epoch 9/10, Average Loss: 2.5245282576634334
Epoch 10/10, Average Loss: 2.4884456414442795
Epoch 1/10, Average Loss: 2.861261477837196
Epoch 2/10, Average Loss: 2.8160387405982386
Epoch 3/10, Average Loss: 2.7721718861506535
Epoch 4/10, Average Loss: 2.7293648316310004
Epoch 5/10, Average Loss: 2.6878035912146934
Epoch 6/10, Average Loss: 2.6476116033700796
Epoch 7/10, Average Loss: 2.6083067123706525
Epoch 8/10, Average Loss: 2.570358555133526
Epoch 9/10, Average Loss: 2.533212691086989
Epoch 10/10, Average Loss: 2.4971622980557955
Epoch 1/10, Average Loss: 2.8514324555030237
Epoch 2/10, Average Loss: 2.8056097764235277
Epoch 3/10, Average Loss: 2.7613452654618484
Epoch 4/10, Average Loss: 2.718218286220844
Epoch 5/10, Aver

Epoch 6/10, Average Loss: 2.3021948997790997
Epoch 7/10, Average Loss: 2.25132863338177
Epoch 8/10, Average Loss: 2.2029834545575655
Epoch 9/10, Average Loss: 2.1572162481454704
Epoch 10/10, Average Loss: 2.113646228496845
Epoch 1/10, Average Loss: 2.596622683451726
Epoch 2/10, Average Loss: 2.5312995250408465
Epoch 3/10, Average Loss: 2.4690480635716363
Epoch 4/10, Average Loss: 2.4101409068474404
Epoch 5/10, Average Loss: 2.3540736400164093
Epoch 6/10, Average Loss: 2.300919872063857
Epoch 7/10, Average Loss: 2.2504324729625997
Epoch 8/10, Average Loss: 2.202598608457125
Epoch 9/10, Average Loss: 2.1573933087862454
Epoch 10/10, Average Loss: 2.1142765008486233
Epoch 1/10, Average Loss: 1.3489388548410857
Epoch 2/10, Average Loss: 1.1322922156407282
Epoch 3/10, Average Loss: 1.0952884710752047
Epoch 4/10, Average Loss: 1.0843665113815895
Epoch 5/10, Average Loss: 1.0742690168894253
Epoch 6/10, Average Loss: 1.0687424100362337
Epoch 7/10, Average Loss: 1.0664137244224547
Epoch 8/10, Av

Epoch 9/10, Average Loss: 0.9610523553994986
Epoch 10/10, Average Loss: 0.9537254012548007
Epoch 1/10, Average Loss: 1.3847307792076697
Epoch 2/10, Average Loss: 1.0641655472608713
Epoch 3/10, Average Loss: 1.0299801322130056
Epoch 4/10, Average Loss: 1.0157850577281071
Epoch 5/10, Average Loss: 0.9986956623884348
Epoch 6/10, Average Loss: 0.9876091590294471
Epoch 7/10, Average Loss: 0.9736648009373592
Epoch 8/10, Average Loss: 0.9543895327127897
Epoch 9/10, Average Loss: 0.9520569627101605
Epoch 10/10, Average Loss: 0.9329008515064533
Epoch 1/10, Average Loss: 1.3935838360052841
Epoch 2/10, Average Loss: 1.0741764939748324
Epoch 3/10, Average Loss: 1.0456100482207078
Epoch 4/10, Average Loss: 1.0208001650296725
Epoch 5/10, Average Loss: 1.0072918167481055
Epoch 6/10, Average Loss: 0.9972318401703468
Epoch 7/10, Average Loss: 0.98360428076524
Epoch 8/10, Average Loss: 0.9784106969833374
Epoch 9/10, Average Loss: 0.9635108049099262
Epoch 10/10, Average Loss: 0.9554079495943509
Epoch 1/1

Epoch 2/10, Average Loss: 1.3356910668886626
Epoch 3/10, Average Loss: 1.2033888248296885
Epoch 4/10, Average Loss: 1.1556966891655556
Epoch 5/10, Average Loss: 1.1272161804712735
Epoch 6/10, Average Loss: 1.107432693701524
Epoch 7/10, Average Loss: 1.0941253680449265
Epoch 8/10, Average Loss: 1.0859645632597117
Epoch 9/10, Average Loss: 1.0729814052581788
Epoch 10/10, Average Loss: 1.0669129646741426
Epoch 1/10, Average Loss: 2.000176246349628
Epoch 2/10, Average Loss: 1.3302725003315852
Epoch 3/10, Average Loss: 1.202303592975323
Epoch 4/10, Average Loss: 1.1557999812639677
Epoch 5/10, Average Loss: 1.1278955101966859
Epoch 6/10, Average Loss: 1.1076708179253798
Epoch 7/10, Average Loss: 1.0930078882437486
Epoch 8/10, Average Loss: 1.0829213793461139
Epoch 9/10, Average Loss: 1.0746396349026606
Epoch 10/10, Average Loss: 1.0684583893189064
Epoch 1/10, Average Loss: 1.9926766047110924
Epoch 2/10, Average Loss: 1.3251279867612398
Epoch 3/10, Average Loss: 1.198087988449977
Epoch 4/10, 

Epoch 5/10, Average Loss: 1.2023435922769399
Epoch 6/10, Average Loss: 1.1722310249622052
Epoch 7/10, Average Loss: 1.1503324453647321
Epoch 8/10, Average Loss: 1.1344991262142474
Epoch 9/10, Average Loss: 1.1231800620372479
Epoch 10/10, Average Loss: 1.1143469874675458
Epoch 1/10, Average Loss: 1.8186622032752404
Epoch 2/10, Average Loss: 1.3518127643145048
Epoch 3/10, Average Loss: 1.2738560548195472
Epoch 4/10, Average Loss: 1.2316088383014385
Epoch 5/10, Average Loss: 1.1960650829168467
Epoch 6/10, Average Loss: 1.1669766802054184
Epoch 7/10, Average Loss: 1.1444365886541514
Epoch 8/10, Average Loss: 1.1279401485736553
Epoch 9/10, Average Loss: 1.1160940803014314
Epoch 10/10, Average Loss: 1.1072370703403767
Epoch 1/10, Average Loss: 2.338992584668673
Epoch 2/10, Average Loss: 2.183067560195923
Epoch 3/10, Average Loss: 2.0453278449865486
Epoch 4/10, Average Loss: 1.9246803613809438
Epoch 5/10, Average Loss: 1.8198629067494319
Epoch 6/10, Average Loss: 1.73008283835191
Epoch 7/10, 

Epoch 8/10, Average Loss: 1.8262273770112258
Epoch 9/10, Average Loss: 1.7568903317818274
Epoch 10/10, Average Loss: 1.6919164785972007
Epoch 1/10, Average Loss: 2.5542069288400504
Epoch 2/10, Average Loss: 2.3926447354830227
Epoch 3/10, Average Loss: 2.2618393164414625
Epoch 4/10, Average Loss: 2.1523358271672177
Epoch 5/10, Average Loss: 2.0559492129545944
Epoch 6/10, Average Loss: 1.9692789554595946
Epoch 7/10, Average Loss: 1.8892490423642672
Epoch 8/10, Average Loss: 1.814437745167659
Epoch 9/10, Average Loss: 1.7448922267326943
Epoch 10/10, Average Loss: 1.679707749073322
Epoch 1/10, Average Loss: 2.5500885009765626
Epoch 2/10, Average Loss: 2.3921056343958926
Epoch 3/10, Average Loss: 2.2642305337465727
Epoch 4/10, Average Loss: 2.156767520537743
Epoch 5/10, Average Loss: 2.062502617102403
Epoch 6/10, Average Loss: 1.9773739209541907
Epoch 7/10, Average Loss: 1.8989682876146756
Epoch 8/10, Average Loss: 1.8258501493013821
Epoch 9/10, Average Loss: 1.7573739070158738
Epoch 10/10,

Epoch 1/10, Average Loss: 1.3598894082582913
Epoch 2/10, Average Loss: 1.1326708243443415
Epoch 3/10, Average Loss: 1.099642616051894
Epoch 4/10, Average Loss: 1.093254790856288
Epoch 5/10, Average Loss: 1.0835661539664636
Epoch 6/10, Average Loss: 1.0733581332060007
Epoch 7/10, Average Loss: 1.0701321739416856
Epoch 8/10, Average Loss: 1.0656121308986957
Epoch 9/10, Average Loss: 1.0560008360789372
Epoch 10/10, Average Loss: 1.044551905301901
Epoch 1/10, Average Loss: 1.3612316021552453
Epoch 2/10, Average Loss: 1.1346695854113653
Epoch 3/10, Average Loss: 1.1105850760753337
Epoch 4/10, Average Loss: 1.0984973219724803
Epoch 5/10, Average Loss: 1.086694531257336
Epoch 6/10, Average Loss: 1.0864079429553106
Epoch 7/10, Average Loss: 1.0798663799579327
Epoch 8/10, Average Loss: 1.06635189973391
Epoch 9/10, Average Loss: 1.062107592362624
Epoch 10/10, Average Loss: 1.049363307769482
Epoch 1/10, Average Loss: 1.3531374271099383
Epoch 2/10, Average Loss: 1.1271739620428818
Epoch 3/10, Aver

In [44]:
grid_results_wine

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Average Accuracy,Training Time,dataset
0,[5],tanh,0.0100,64,10,0.538476,14.801621,wine_quality
1,[5],tanh,0.0010,64,10,0.414777,41.167479,wine_quality
2,[5],tanh,0.0001,64,10,0.217970,33.789345,wine_quality
3,[5],relu,0.0100,64,10,0.542130,13.735649,wine_quality
4,[5],relu,0.0010,64,10,0.428632,24.834079,wine_quality
5,[5],relu,0.0001,64,10,0.052520,19.730580,wine_quality
6,[5],sigmoid,0.0100,64,10,0.538862,15.396676,wine_quality
7,[5],sigmoid,0.0010,64,10,0.492690,17.701936,wine_quality
8,[5],sigmoid,0.0001,64,10,0.035206,20.762974,wine_quality
9,[10],tanh,0.0100,64,10,0.550788,22.377950,wine_quality


### Congressional Voting

In [45]:
hidden_layer_sizes_list = [[5],[10],[25, 30], [20, 25, 30]]
activation_functions = [F.tanh, F.relu, F.sigmoid]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [64]
num_epochs_list = [10]

dataset = cong_voting

train_X, train_Y, test_X, test_Y = train_test_split(dataset, "class", return_torch=True)

train_data = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

test_data = TensorDataset(test_X, test_Y)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

input_size = train_X.shape[1]

num_classes = 2

grid_results_voting, best_accuracy, best_combination = grid_search_cv(
    hidden_layer_sizes_list, activation_functions, learning_rates, batch_sizes, num_epochs_list, train_loader,
    k_folds=5, use_scaling=True)

grid_results_voting['dataset'] = 'cong_voting'

Epoch 1/10, Average Loss: 1.074934720993042
Epoch 2/10, Average Loss: 0.9035237431526184
Epoch 3/10, Average Loss: 0.7592998544375101
Epoch 4/10, Average Loss: 0.5777889887491862
Epoch 5/10, Average Loss: 0.5432572960853577
Epoch 6/10, Average Loss: 0.40527820587158203
Epoch 7/10, Average Loss: 0.3490559160709381
Epoch 8/10, Average Loss: 0.3456357419490814
Epoch 9/10, Average Loss: 0.3522816201051076
Epoch 10/10, Average Loss: 0.2685555915037791
Epoch 1/10, Average Loss: 1.0212214589118958
Epoch 2/10, Average Loss: 0.8432358304659525
Epoch 3/10, Average Loss: 0.6551553805669149
Epoch 4/10, Average Loss: 0.6221447785695394
Epoch 5/10, Average Loss: 0.47259072462717694
Epoch 6/10, Average Loss: 0.4194834033648173
Epoch 7/10, Average Loss: 0.3748304645220439
Epoch 8/10, Average Loss: 0.300258606672287
Epoch 9/10, Average Loss: 0.3336060643196106
Epoch 10/10, Average Loss: 0.2684950033823649
Epoch 1/10, Average Loss: 1.0098368724187214
Epoch 2/10, Average Loss: 0.8441717823346456
Epoch 3/

Epoch 3/10, Average Loss: 0.8574781020482382
Epoch 4/10, Average Loss: 0.6992871960004171
Epoch 5/10, Average Loss: 0.5900222063064575
Epoch 6/10, Average Loss: 0.5469769636789957
Epoch 7/10, Average Loss: 0.4444561501344045
Epoch 8/10, Average Loss: 0.44097548723220825
Epoch 9/10, Average Loss: 0.44444485505421955
Epoch 10/10, Average Loss: 0.41404545307159424
Epoch 1/10, Average Loss: 1.6019543011983235
Epoch 2/10, Average Loss: 1.1828236977259319
Epoch 3/10, Average Loss: 0.9146458705266317
Epoch 4/10, Average Loss: 0.7732807397842407
Epoch 5/10, Average Loss: 0.6029676993687948
Epoch 6/10, Average Loss: 0.5464995702107748
Epoch 7/10, Average Loss: 0.5139149924119314
Epoch 8/10, Average Loss: 0.47785648703575134
Epoch 9/10, Average Loss: 0.4155654013156891
Epoch 10/10, Average Loss: 0.3979838689168294
Epoch 1/10, Average Loss: 1.4963870445887248
Epoch 2/10, Average Loss: 1.5534661213556926
Epoch 3/10, Average Loss: 1.5189483960469563
Epoch 4/10, Average Loss: 1.519816279411316
Epoch

Epoch 6/10, Average Loss: 0.7556566993395487
Epoch 7/10, Average Loss: 0.7366400758425394
Epoch 8/10, Average Loss: 0.7228436668713888
Epoch 9/10, Average Loss: 0.7316259543100992
Epoch 10/10, Average Loss: 0.7370315194129944
Epoch 1/10, Average Loss: 0.7642275094985962
Epoch 2/10, Average Loss: 0.774961531162262
Epoch 3/10, Average Loss: 0.7676618099212646
Epoch 4/10, Average Loss: 0.7575567960739136
Epoch 5/10, Average Loss: 0.7624529997507731
Epoch 6/10, Average Loss: 0.7466118733088175
Epoch 7/10, Average Loss: 0.7325563033421835
Epoch 8/10, Average Loss: 0.7258973320325216
Epoch 9/10, Average Loss: 0.7390031417210897
Epoch 10/10, Average Loss: 0.7294008135795593
Epoch 1/10, Average Loss: 0.7773540814717611
Epoch 2/10, Average Loss: 0.7768575151761373
Epoch 3/10, Average Loss: 0.7606993714968363
Epoch 4/10, Average Loss: 0.748020072778066
Epoch 5/10, Average Loss: 0.7560543815294901
Epoch 6/10, Average Loss: 0.7620214819908142
Epoch 7/10, Average Loss: 0.7435738841692606
Epoch 8/10

Epoch 1/10, Average Loss: 1.2178582350413005
Epoch 2/10, Average Loss: 1.242900013923645
Epoch 3/10, Average Loss: 1.0582552750905354
Epoch 4/10, Average Loss: 1.113172133763631
Epoch 5/10, Average Loss: 1.181827425956726
Epoch 6/10, Average Loss: 1.0764343738555908
Epoch 7/10, Average Loss: 1.2710209488868713
Epoch 8/10, Average Loss: 1.1692362626393635
Epoch 9/10, Average Loss: 1.044761021931966
Epoch 10/10, Average Loss: 1.0849289894104004
Epoch 1/10, Average Loss: 1.0179946223894756
Epoch 2/10, Average Loss: 1.1137080987294514
Epoch 3/10, Average Loss: 1.0137309829394023
Epoch 4/10, Average Loss: 1.076917787392934
Epoch 5/10, Average Loss: 1.1508311827977498
Epoch 6/10, Average Loss: 1.0967390537261963
Epoch 7/10, Average Loss: 1.062780539194743
Epoch 8/10, Average Loss: 1.1569926540056865
Epoch 9/10, Average Loss: 1.0983474850654602
Epoch 10/10, Average Loss: 1.056156595547994
Epoch 1/10, Average Loss: 1.086870511372884
Epoch 2/10, Average Loss: 1.1467430194218953
Epoch 3/10, Aver

Epoch 5/10, Average Loss: 1.2843409776687622
Epoch 6/10, Average Loss: 1.3943788210550945
Epoch 7/10, Average Loss: 1.1769104798634846
Epoch 8/10, Average Loss: 1.117808997631073
Epoch 9/10, Average Loss: 1.1212991674741108
Epoch 10/10, Average Loss: 1.1496059099833171
Epoch 1/10, Average Loss: 1.246869643529256
Epoch 2/10, Average Loss: 1.2551863590876262
Epoch 3/10, Average Loss: 1.2815628846486409
Epoch 4/10, Average Loss: 1.2679448525110881
Epoch 5/10, Average Loss: 1.2889378468195598
Epoch 6/10, Average Loss: 1.207937479019165
Epoch 7/10, Average Loss: 1.2163021167119343
Epoch 8/10, Average Loss: 1.185759921868642
Epoch 9/10, Average Loss: 1.2822513977686565
Epoch 10/10, Average Loss: 1.2540510098139446
Epoch 1/10, Average Loss: 0.8296816547711691
Epoch 2/10, Average Loss: 0.6502251823743185
Epoch 3/10, Average Loss: 0.5100051462650299
Epoch 4/10, Average Loss: 0.44470328092575073
Epoch 5/10, Average Loss: 0.40199777483940125
Epoch 6/10, Average Loss: 0.42350005110104877
Epoch 7/1

Epoch 8/10, Average Loss: 0.009328079875558615
Epoch 9/10, Average Loss: 0.007221865855778257
Epoch 10/10, Average Loss: 0.008968418464064598
Epoch 1/10, Average Loss: 0.22931899627049765
Epoch 2/10, Average Loss: 0.09055064494411151
Epoch 3/10, Average Loss: 0.04827915628751119
Epoch 4/10, Average Loss: 0.03182208010305961
Epoch 5/10, Average Loss: 0.024392186353604
Epoch 6/10, Average Loss: 0.03084071632474661
Epoch 7/10, Average Loss: 0.01100772456265986
Epoch 8/10, Average Loss: 0.0085750970368584
Epoch 9/10, Average Loss: 0.009567616507411003
Epoch 10/10, Average Loss: 0.00544495116143177
Epoch 1/10, Average Loss: 0.21317797402540842
Epoch 2/10, Average Loss: 0.12740161269903183
Epoch 3/10, Average Loss: 0.05396617700656255
Epoch 4/10, Average Loss: 0.05072606975833575
Epoch 5/10, Average Loss: 0.030644388558963936
Epoch 6/10, Average Loss: 0.0211955316675206
Epoch 7/10, Average Loss: 0.01371908358608683
Epoch 8/10, Average Loss: 0.023521432653069496
Epoch 9/10, Average Loss: 0.00

Epoch 7/10, Average Loss: 0.016424630690986913
Epoch 8/10, Average Loss: 0.009597384370863438
Epoch 9/10, Average Loss: 0.013223648187704384
Epoch 10/10, Average Loss: 0.004862031016576414
Epoch 1/10, Average Loss: 0.36325215299924213
Epoch 2/10, Average Loss: 0.3537360429763794
Epoch 3/10, Average Loss: 0.29046958684921265
Epoch 4/10, Average Loss: 0.24094334741433462
Epoch 5/10, Average Loss: 0.23778333266576132
Epoch 6/10, Average Loss: 0.22987321515878043
Epoch 7/10, Average Loss: 0.16381235172351202
Epoch 8/10, Average Loss: 0.14133363217115402
Epoch 9/10, Average Loss: 0.1263921931385994
Epoch 10/10, Average Loss: 0.11884483695030212
Epoch 1/10, Average Loss: 0.3715396523475647
Epoch 2/10, Average Loss: 0.32577647268772125
Epoch 3/10, Average Loss: 0.33354486028353375
Epoch 4/10, Average Loss: 0.24687371154626211
Epoch 5/10, Average Loss: 0.22001657883326212
Epoch 6/10, Average Loss: 0.210517684618632
Epoch 7/10, Average Loss: 0.1881634642680486
Epoch 8/10, Average Loss: 0.172533

Epoch 8/10, Average Loss: 0.5993027885754904
Epoch 9/10, Average Loss: 0.5528795520464579
Epoch 10/10, Average Loss: 0.5358004172643026
Epoch 1/10, Average Loss: 0.6564445296923319
Epoch 2/10, Average Loss: 0.6422505776087443
Epoch 3/10, Average Loss: 0.6045092542966207
Epoch 4/10, Average Loss: 0.6072926719983419
Epoch 5/10, Average Loss: 0.5894335508346558
Epoch 6/10, Average Loss: 0.5789460937182108
Epoch 7/10, Average Loss: 0.5602234601974487
Epoch 8/10, Average Loss: 0.5665659705797831
Epoch 9/10, Average Loss: 0.5334320267041525
Epoch 10/10, Average Loss: 0.519227663675944
Epoch 1/10, Average Loss: 0.6577333807945251
Epoch 2/10, Average Loss: 0.6330110430717468
Epoch 3/10, Average Loss: 0.6282989581425985
Epoch 4/10, Average Loss: 0.5764060417811075
Epoch 5/10, Average Loss: 0.588047464688619
Epoch 6/10, Average Loss: 0.5663332541783651
Epoch 7/10, Average Loss: 0.5971038540204366
Epoch 8/10, Average Loss: 0.5611674388249716
Epoch 9/10, Average Loss: 0.590655525525411
Epoch 10/10

Epoch 9/10, Average Loss: 0.7381816705067953
Epoch 10/10, Average Loss: 0.7209096550941467
Epoch 1/10, Average Loss: 0.9060085217157999
Epoch 2/10, Average Loss: 0.931769867738088
Epoch 3/10, Average Loss: 0.906456708908081
Epoch 4/10, Average Loss: 0.8330294291178385
Epoch 5/10, Average Loss: 0.8937829732894897
Epoch 6/10, Average Loss: 0.8219660917917887
Epoch 7/10, Average Loss: 0.8014228741327921
Epoch 8/10, Average Loss: 0.8226561943689982
Epoch 9/10, Average Loss: 0.7545611262321472
Epoch 10/10, Average Loss: 0.7774272163709005
Epoch 1/10, Average Loss: 0.9423375129699707
Epoch 2/10, Average Loss: 0.9453672766685486
Epoch 3/10, Average Loss: 0.8825875918070475
Epoch 4/10, Average Loss: 0.9364376664161682
Epoch 5/10, Average Loss: 0.7992695172627767
Epoch 6/10, Average Loss: 0.804831326007843
Epoch 7/10, Average Loss: 0.8524681131045023
Epoch 8/10, Average Loss: 0.8103073438008627
Epoch 9/10, Average Loss: 0.8120043675104777
Epoch 10/10, Average Loss: 0.7591289679209391
Epoch 1/10

Epoch 1/10, Average Loss: 1.0401983658472698
Epoch 2/10, Average Loss: 1.0408274332682292
Epoch 3/10, Average Loss: 0.9150534272193909
Epoch 4/10, Average Loss: 1.0867724816004436
Epoch 5/10, Average Loss: 0.897241493066152
Epoch 6/10, Average Loss: 0.8715798457463583
Epoch 7/10, Average Loss: 0.9230287273724874
Epoch 8/10, Average Loss: 0.8771918018658956
Epoch 9/10, Average Loss: 0.8087437947591146
Epoch 10/10, Average Loss: 0.8204637169837952
Epoch 1/10, Average Loss: 0.6655864119529724
Epoch 2/10, Average Loss: 0.6856606801350912
Epoch 3/10, Average Loss: 0.6309844255447388
Epoch 4/10, Average Loss: 0.6040470997492472
Epoch 5/10, Average Loss: 0.5299461583296458
Epoch 6/10, Average Loss: 0.48230822881062824
Epoch 7/10, Average Loss: 0.3826008439064026
Epoch 8/10, Average Loss: 0.2876925120751063
Epoch 9/10, Average Loss: 0.21149984498818716
Epoch 10/10, Average Loss: 0.18242262303829193
Epoch 1/10, Average Loss: 0.6799221436182658
Epoch 2/10, Average Loss: 0.6766951282819113
Epoch 

In [46]:
grid_results_voting

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Average Accuracy,Training Time,dataset
0,[5],tanh,0.0100,64,10,0.896471,1.121827,cong_voting
1,[5],tanh,0.0010,64,10,0.287731,1.068501,cong_voting
2,[5],tanh,0.0001,64,10,0.212773,0.878100,cong_voting
3,[5],relu,0.0100,64,10,0.890756,0.844678,cong_voting
4,[5],relu,0.0010,64,10,0.304706,0.628949,cong_voting
5,[5],relu,0.0001,64,10,0.235798,0.704427,cong_voting
6,[5],sigmoid,0.0100,64,10,0.913782,0.665610,cong_voting
7,[5],sigmoid,0.0010,64,10,0.264706,0.610235,cong_voting
8,[5],sigmoid,0.0001,64,10,0.166891,0.686372,cong_voting
9,[10],tanh,0.0100,64,10,0.936639,0.750672,cong_voting


#### Merging

In [48]:
full_results_df = pd.concat([grid_results_voting,grid_results_wine,grid_results_bank], ignore_index=True)
full_results_df

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Average Accuracy,Training Time,dataset
0,[5],tanh,0.0100,64,10,0.896471,1.121827,cong_voting
1,[5],tanh,0.0010,64,10,0.287731,1.068501,cong_voting
2,[5],tanh,0.0001,64,10,0.212773,0.878100,cong_voting
3,[5],relu,0.0100,64,10,0.890756,0.844678,cong_voting
4,[5],relu,0.0010,64,10,0.304706,0.628949,cong_voting
...,...,...,...,...,...,...,...,...
103,"[20, 25, 30]",relu,0.0010,64,10,0.897481,108.624659,bank_marketing
104,"[20, 25, 30]",relu,0.0001,64,10,0.897572,120.566443,bank_marketing
105,"[20, 25, 30]",sigmoid,0.0100,64,10,0.898968,121.211843,bank_marketing
106,"[20, 25, 30]",sigmoid,0.0010,64,10,0.898483,146.621264,bank_marketing


In [49]:
full_results_df.to_csv('cv_grid_search_results.csv', index=False)

### Top Perfroming Configurations

In [50]:
top_models_rows = []

for dataset in full_results_df['dataset'].unique():
    top_models_rows.extend(full_results_df[full_results_df['dataset'] == dataset].nlargest(2, 'Average Accuracy').iterrows())

top_models_rows_data = [row[1] for row in top_models_rows]

top_models_df = pd.DataFrame(top_models_rows_data).reset_index(drop=True)

top_models_df

,Hidden Layer Sizes,Activation Function,Learning Rate,Batch Size,Number of Epochs,Average Accuracy,Training Time,dataset
0,"[25, 30]",tanh,0.010,64,10,0.965546,1.279610,cong_voting
1,"[25, 30]",relu,0.010,64,10,0.965378,0.975561,cong_voting
2,"[20, 25, 30]",relu,0.010,64,10,0.561176,19.611423,wine_quality
3,"[20, 25, 30]",tanh,0.010,64,10,0.553872,22.225502,wine_quality
4,[5],sigmoid,0.010,64,10,0.899545,13.074883,bank_marketing
5,[10],sigmoid,0.001,64,10,0.899545,10.614720,bank_marketing
